# Image Alignment - CV Project 
## Giulia Benvenuto - 4678610
- The detailed description of the entire notebook is in the report in the .zip file. 
- I cleared both the output cells in the notebook and the output files from the "Outputs" folder in order to reduce the size of the .zip file. 

## Image Alignment
(Detailed description in the Sections 1 and 2 of the Report)

This project is about Image alignment, also known as image registration, which is a computer vision technique aimed (as the name suggests) to aligning two or more images so that corresponding points or features in the images are spatially matched.

## Image Alignment Pipeline
(Detailed description in the Section 2 of the Report)
1. Features detection + Features description
2. Feature matching
3. Find the homography matrix
4. Warp perspective

Here I decided to:
- Use SIFT to localize and describe the keypoints because this algorithm is consideres to be very robust even if its computational time, compared to other algorithms, could be longer.
- Use a Brute Force matcher to perform feature matching, because it exhaustively searches for the best matches and it guarantees accurate results.

## Import the libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Utility functions
(Detailed description in the Section 7 of the Report)

In [ ]:
def plot_2 (img1, img2, title1, title2):
    fig = plt.figure(figsize=(30, 70))
    fig.add_subplot(1,2,1)
    plt.title(title1)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    fig.add_subplot(1,2,2)
    plt.title(title2)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def plot_3 (img1, img2, img3, title1, title2, title3):
    fig = plt.figure(figsize=(30, 70)) 
    fig.add_subplot(1,3,1)
    plt.title(title1)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    fig.add_subplot(1,3,2)
    plt.title(title2)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    fig.add_subplot(1,3,3)
    plt.title(title3)
    plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def featureMatching(des_1, des_2, gl, g):
    # Matching
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_1, des_2, k=2)
    
    n_of_matches = len(matches)
    print("Total number of matches:", n_of_matches)
    
    # Apply ratio test
    for m,n in matches:
        if m.distance < 0.5 * n.distance:
            gl.append([m])
            g.append(m)
    
    print("Number of good matches:", len(good))
    n_of_good_matches  = len(good)
    
    return good_list, good

In [ ]:
def findHomography(n_g_matches, g, kp_1, kp_2):
    MIN_MATCH_COUNT = 10
    
    if n_g_matches > MIN_MATCH_COUNT:
        src_pts = np.float32([ kp_1[m.queryIdx].pt for m in g ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp_2[m.trainIdx].pt for m in g ]).reshape(-1,1,2)
    
    homo, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    print("Homography matrix:\n", homo)
    return homo

## Poster, Test 1 - 2D rotation of 90 degrees - SUCCESS
(Detailed description in the Section 3.1 of the Report)

### Read the images

In [ ]:
# Reference image:
img_poster = cv2.imread('Photos/Poster/img_1.jpeg')

In [ ]:
img_test1 = cv2.imread('Photos/Poster/img_2.jpeg')

# Convert the images to gray scale:
img_poster = cv2.cvtColor(img_poster, cv2.COLOR_BGR2GRAY)
img_test1 = cv2.cvtColor(img_test1, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_poster, img_test1, "Poster", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_poster, None)
kp2, des2 = sift.detectAndCompute(img_test1, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_poster = None
img_sift_poster = cv2.drawKeypoints(img_poster, kp1, img_sift_poster, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_poster/sift_poster.jpg', img_sift_poster)

# Draw the keypoints on the test image:
img_sift_test1 = None
img_sift_test1 = cv2.drawKeypoints(img_test1, kp2, img_sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_poster/sift_test1.jpg', img_sift_test1)

# Show the result of keypoint detection:
plot_2(img_sift_poster, img_sift_test1, "Poster keypoints", "Test image keypoints")

### Fetature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_test1 = None
img_match_test1 = cv2.drawMatchesKnn(img_poster, kp1, img_test1, kp2, good_list, img_match_test1, flags=2)

plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_test1, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(img_test1, homography, (img_poster.shape[1], img_poster.shape[0]))
cv2.imwrite('Outputs/Outputs_poster/aligned_img_test1.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(img_poster, img_test1, aligned_img, "Poster", "Test image", "Aligned test image")

## Poster, Test 2 - 3D rotation - SUCCESS
(Detailed description in the Section 3.2 of the Report)

### Read the images

In [ ]:
img_poster = cv2.imread('Photos/Poster/img_1.jpeg')
img_test2 = cv2.imread('Photos/Poster/img_4.jpeg')

# Convert the images to gray scale:
img_poster = cv2.cvtColor(img_poster, cv2.COLOR_BGR2GRAY)
img_test2 = cv2.cvtColor(img_test2, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_poster, img_test2, "Poster", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_poster, None)
kp2, des2 = sift.detectAndCompute(img_test2, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_poster = None
img_sift_poster = cv2.drawKeypoints(img_poster, kp1, img_sift_poster, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_poster/sift_poster.jpg', img_sift_poster)

# Draw the keypoints on the test image:
img_sift_test2 = None
img_sift_test2 = cv2.drawKeypoints(img_test2, kp2, img_sift_test2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_poster/sift_test2.jpg', img_sift_test2)

# Show the result of keypoint detection:
plot_2(img_sift_poster, img_sift_test2, "Poster keypoints", "Test image keypoints")

### Fetature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_test2 = None
img_match_test2 = cv2.drawMatchesKnn(img_poster, kp1, img_test2, kp2, good_list, img_match_test2, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_test2, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(img_test2, homography, (img_poster.shape[1], img_poster.shape[0]))
cv2.imwrite('Outputs/Outputs_poster/aligned_img_test2.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(img_poster, img_test2, aligned_img, "Poster", "Test image", "Aligned test image")

## Drawing 1, Test 1 - 2D rotation of 45 degrees - SUCCESS
(Detailed description in the Section 3.3 of the Report)

### Read the images

In [ ]:
img_draw1 = cv2.imread('Photos/Draw1/img1.jpg')
img_draw1_test1 = cv2.imread('Photos/Draw1/img5.jpg')

# Convert the images to gray scale:
img_draw1 = cv2.cvtColor(img_draw1, cv2.COLOR_BGR2GRAY)
img_draw1_test1 = cv2.cvtColor(img_draw1_test1, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_draw1, img_draw1_test1, "Drawing 1", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_draw1, None)
kp2, des2 = sift.detectAndCompute(img_draw1_test1, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_draw1 = None
img_sift_draw1 = cv2.drawKeypoints(img_draw1, kp1, img_sift_draw1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw1/sift_draw1.jpg', img_sift_draw1)

# Draw the keypoints on the test image:
img_sift_draw1_test1 = None
img_sift_draw1_test1 = cv2.drawKeypoints(img_draw1_test1, kp2, img_sift_draw1_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw1/sift_draw1_test1.jpg', img_sift_draw1_test1)

# Show the result of keypoint detection:
plot_2(img_sift_draw1, img_sift_draw1_test1, "Drawing 1 keypoints", "Test image keypoints")

### Fetature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_draw1_test1 = None
img_match_draw1_test1 = cv2.drawMatchesKnn(img_draw1, kp1, img_draw1_test1, kp2, good_list, img_match_draw1_test1, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_draw1_test1, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(img_draw1_test1, homography, (img_draw1.shape[1], img_draw1.shape[0]))
cv2.imwrite('Outputs/Outputs_draw1/aligned_img_test1.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(img_draw1, img_draw1_test1, aligned_img, "Drawing 1", "Test image", "Aligned test image")

## Drawing 1, Test 2 - 3D rotation - FAIL
(Detailed description in the Section 3.4 of the Report)

### Read the images

In [ ]:
img_draw1 = cv2.imread('Photos/Draw1/img1.jpg')
img_draw1_test2 = cv2.imread('Photos/Draw1/img6.jpg')

# Convert the images to gray scale:
img_draw1 = cv2.cvtColor(img_draw1, cv2.COLOR_BGR2GRAY)
img_draw1_test2 = cv2.cvtColor(img_draw1_test2, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_draw1, img_draw1_test2, "Drawing 1", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_draw1, None)
kp2, des2 = sift.detectAndCompute(img_draw1_test2, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_draw1 = None
img_sift_draw1 = cv2.drawKeypoints(img_draw1, kp1, img_sift_draw1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw1/sift_draw1.jpg', img_sift_draw1)

# Draw the keypoints on the test image:
img_sift_draw1_test2 = None
img_sift_draw1_test2 = cv2.drawKeypoints(img_draw1_test2, kp2, img_sift_draw1_test2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw1/sift_draw1_test2.jpg', img_sift_draw1_test2)

# Show the result of keypoint detection:
plot_2(img_sift_draw1, img_sift_draw1_test2, "Drawing 1 keypoints", "Test image keypoints")

### Fetature Matching

Need to set to 0.75 otherwise we don't have enough good features.

In [ ]:
# Matching
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

n_of_matches = len(matches)
print("Total number of matches:", n_of_matches)

# Apply ratio test
good_list = []
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good_list.append([m])
        good.append(m)

print("Number of good matches:", len(good))
n_of_good_matches  = len(good)

In [ ]:
img_match_draw1_test2 = None
img_match_draw1_test2 = cv2.drawMatchesKnn(img_draw1, kp1, img_draw1_test2, kp2, good_list, img_match_draw1_test2, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_draw1_test2, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(img_draw1_test2, homography, (img_draw1.shape[1], img_draw1.shape[0]))
cv2.imwrite('Outputs/Outputs_draw1/aligned_img_test2.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(img_draw1, img_draw1_test2, aligned_img, "Drawing 1", "Test image", "Aligned test image")

## Drawing 2, Test 1 - 3D rotation - FAIL
(Detailed description in the Section 3.5 of the Report)

### Read the images

In [ ]:
img_draw2 = cv2.imread('Photos/Draw2/img1.jpg')
img_draw2_test1 = cv2.imread('Photos/Draw2/img3.jpg')

# Convert the images to gray scale:
img_draw2 = cv2.cvtColor(img_draw2, cv2.COLOR_BGR2GRAY)
img_draw2_test1 = cv2.cvtColor(img_draw2_test1, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_draw2, img_draw2_test1, "Drawing 2", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_draw2, None)
kp2, des2 = sift.detectAndCompute(img_draw2_test1, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_draw2 = None
img_sift_draw2 = cv2.drawKeypoints(img_draw2, kp1, img_sift_draw2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw2/sift_draw2.jpg', img_sift_draw2)

# Draw the keypoints on the test image:
img_sift_draw2_test1 = None
img_sift_draw2_test1 = cv2.drawKeypoints(img_draw2_test1, kp2, img_sift_draw2_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_draw2/sift_draw2_test1.jpg', img_sift_draw2_test1)

# Show the result of keypoint detection:
plot_2(img_sift_draw2, img_sift_draw2_test1, "Drawing 2 keypoints", "Test image keypoints")

### Fetature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_draw2_test1 = None
img_match_draw2_test1 = cv2.drawMatchesKnn(img_draw2, kp1, img_draw2_test1, kp2, good_list, img_match_draw2_test1, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_draw2_test1, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(img_draw2_test1, homography, (img_draw2.shape[1], img_draw2.shape[0]))
cv2.imwrite('Outputs/Outputs_draw2/aligned_img_test1.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(img_draw2, img_draw2_test1, aligned_img, "Drawing 2", "Test image", "Aligned test image")

## 3D Object - Rubik's Cube - FAIL
(Detailed description in the Section 3.6 of the Report)

### Read the images

In [ ]:
rubik = cv2.imread('Photos/3D_objects/rubik_6.jpeg')
rubik_test = cv2.imread('Photos/3D_objects/rubik_4.jpeg')

# Convert the images to gray scale:
rubik = cv2.cvtColor(rubik, cv2.COLOR_BGR2GRAY)
rubik_test = cv2.cvtColor(rubik_test, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(rubik, rubik_test, "Rubik's Cube", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(rubik, None)
kp2, des2 = sift.detectAndCompute(rubik_test, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
rubik_sift = None
rubik_sift = cv2.drawKeypoints(rubik, kp1, rubik_sift, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_rubik/rubik_sift.jpg', rubik_sift)

# Draw the keypoints on the test image:
rubik_sift_test = None
rubik_sift_test = cv2.drawKeypoints(rubik_test, kp2, rubik_sift_test, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_rubik/rubik_sift_test.jpg', rubik_sift_test)

# Show the result of keypoint detection:
plot_2(rubik_sift, rubik_sift_test, "Rubik keypoints", "Test image keypoints")

### Feature Matching

Need to set to 0.75 otherwise we don't have enough goof features.

In [ ]:
# Matching
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

n_of_matches = len(matches)
print("Total number of matches:", n_of_matches)

# Apply ratio test
good_list = []
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good_list.append([m])
        good.append(m)

print("Number of good matches:", len(good))
n_of_good_matches  = len(good)

In [ ]:
match_rubik = None
match_rubik = cv2.drawMatchesKnn(rubik, kp1, rubik_test, kp2, good_list, match_rubik, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(match_rubik, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp perspective

In [ ]:
aligned_img = cv2.warpPerspective(rubik_test, homography, (rubik.shape[1], rubik.shape[0]))
cv2.imwrite('Outputs/Outputs_rubik/aligned_rubik.jpg', aligned_img)

### Results

In [ ]:
# Show the result
plot_3(rubik, rubik_test, aligned_img, "Rubik", "Test image", "Aligned test image")

## 3D Object - Canteen - FAIL
(Detailed description in the Section 3.7 of the Report)

### Read the images

In [ ]:
canteen = cv2.imread('Photos/3D_objects/canteen_1.jpeg')
canteen_test = cv2.imread('Photos/3D_objects/canteen_2.jpeg')

# Convert the images to gray scale:
canteen = cv2.cvtColor(canteen, cv2.COLOR_BGR2GRAY)
canteen_test = cv2.cvtColor(canteen_test, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(canteen, canteen_test, "Canteen", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(canteen, None)
kp2, des2 = sift.detectAndCompute(canteen_test, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_canteen= None
img_sift_canteen = cv2.drawKeypoints(canteen, kp1, img_sift_canteen, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_canteen/sift_canteen.jpg', img_sift_canteen)

# Draw the keypoints on the test image:
img_sift_test = None
img_sift_test = cv2.drawKeypoints(canteen_test, kp2, img_sift_test, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_canteen/sift_canteen_test.jpg', img_sift_test)

# Show the result of keypoint detection:
plot_2(img_sift_canteen, img_sift_test, "Canteen keypoints", "Test image keypoints")

### Feature Matching

Need to set to 0.75 otherwise we don't have enough goof features.

In [ ]:
# Matching
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

n_of_matches = len(matches)
print("Total number of matches:", n_of_matches)

# Apply ratio test
good_list = []
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good_list.append([m])
        good.append(m)

print("Number of good matches:", len(good))
n_of_good_matches  = len(good)

In [ ]:
canteen_match = None
canteen_match = cv2.drawMatchesKnn(canteen, kp1, canteen_test, kp2, good_list, canteen_match, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(canteen_match, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img_canteen = cv2.warpPerspective(canteen_test, homography, (canteen.shape[1], canteen.shape[0]))
cv2.imwrite('Outputs/Outputs_canteen/aligned_canteen.jpg', aligned_img_canteen)

### Results

In [ ]:
# Show the result
plot_3(canteen, canteen_test, aligned_img_canteen, "Canteen", "Test image", "Aligned test image")

## Image alignment for form alignment - 3D rotation - SUCCESS
(Detailed description in the Section 4.1 of the Report)

### Read the images

In [ ]:
img_template = cv2.imread('Photos/Form/form_template.jpg')
img_test = cv2.imread('Photos/Form/img12.jpeg')

# Convert the images to gray scale:
img_template = cv2.cvtColor(img_template, cv2.COLOR_BGR2GRAY)
img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(img_template, img_test, "Form template", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(img_template, None)
kp2, des2 = sift.detectAndCompute(img_test, None)

print("Number of keypoints poster image:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
img_sift_template = None
img_sift_template = cv2.drawKeypoints(img_template, kp1, img_sift_template, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_form/sift_form_template.jpg', img_sift_template)

# Draw the keypoints on the test image:
img_sift_test = None
img_sift_test = cv2.drawKeypoints(img_test, kp2, img_sift_test, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_form/sift_test.jpg', img_sift_test)

# Show the result of keypoint detection:
plot_2(img_sift_template, img_sift_test, "Form template keypoints", "Test image keypoints")

### Fetature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_form = None
img_match_form = cv2.drawMatchesKnn(img_template, kp1, img_test, kp2, good_list, img_match_form, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_form, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography 

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp Perspective

In [ ]:
aligned_img_form = cv2.warpPerspective(img_test, homography, (img_template.shape[1], img_template.shape[0]))
cv2.imwrite('Outputs_form/aligned_img_form.jpg', aligned_img_form)

### Results

In [ ]:
# Show the result
plot_3(img_template, img_test, aligned_img_form, "Form template", "Test image", "Aligned test image")

## Image alignment for panorama stitching from scratch
(Detailed description in the Section 4.2.1 of the Report)

### Read the images

In [ ]:
# Load images:
left_img = cv2.imread('Photos/Bedroom/try1_4.jpg')
right_img = cv2.imread('Photos/Bedroom/try1_5.jpg')

# Convert the images to gray scale:
left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(left_img, right_img, "Left image", "Right image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp_left, des_left = sift.detectAndCompute(left_img, None)
kp_right, des_right = sift.detectAndCompute(right_img, None)

print("Number of keypoints left image:", len(kp_left))
print("Number of keypoints right image:", len(kp_right))

In [ ]:
# Draw the keypoints on the left image:
left_img_sift = None
left_img_sift = cv2.drawKeypoints(left_img, kp_left, left_img_sift, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_stitching/left_img_sift.jpg', left_img_sift)

# Draw the keypoints on the right image:
right_img_sift = None
right_img_sift = cv2.drawKeypoints(right_img, kp_right, right_img_sift, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_stitching/right_img_sift.jpg', right_img_sift)

# Show the result of keypoint detection:
plot_2(left_img_sift, right_img_sift, "Left image keypoints", "Right image keypoints")

### Feature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des_left, des_right, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
img_match_bedroom = None
img_match_bedroom = cv2.drawMatchesKnn(left_img, kp_left, right_img, kp_right, good_list, img_match_bedroom, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(img_match_bedroom, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography

In [ ]:
homography = findHomography(n_of_good_matches, good, kp_left, kp_right)

### Stitch Images

In [ ]:
# https://github.com/bakkyn/Image-Alignment-using-OpenCV/blob/main/image_alignment/Image_Alignment.ipynb
def stitchImages(img_left, img_right, homography):
    
    # get height and weight values
    rowl, coll = img_left.shape[:2]
    rowr, colr = img_right.shape[:2]

    # left to right - corner coordinates
    left = np.float32([[0,0], [0, rowl],[coll, rowl], [coll, 0]]).reshape(-1, 1,2)
    right = np.float32([[0,0], [0,rowr], [colr,rowr], [colr,0]]).reshape(-1,1,2)

    # To apply homography we need wrap perspective. Calculate the transformation matrix
    right = cv2.perspectiveTransform(right, homography)
        
    points = np.concatenate((left, right), axis=0)

    # get min max coordinates 
    min_p = points.min(axis=0).ravel() # search for one dim
    max_p = points.max(axis=0).ravel()
    [minx, miny] = np.int32(min_p - 0.5)
    [maxx, maxy] = np.int32(max_p + 0.5)
  
    block = [-minx, -miny]
    hom_translation = np.array([[1, 0, block[0]], [0, 1, block[1]], [0, 0, 1]])

    final_img = cv2.warpPerspective(img_right, hom_translation.dot(homography), (maxx-minx, maxy-miny))
    final_img[block[1]:rowl+block[1], block[0]:coll+block[0]] = img_left

    return final_img

### Result

In [ ]:
result = stitchImages(left_img, right_img, homography)

plt.title("Result image")
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.show()

## Image alignment for panorama using OpenCV function - 2 Images
(Detailed description in the Section 4.2.2 of the Report)

### Read the images

In [ ]:
# Load images:
left_img = cv2.imread('Photos/Bedroom/try1_4.jpg')
right_img = cv2.imread('Photos/Bedroom/try1_5.jpg')

# Convert the images to gray scale:
left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(left_img, right_img, "Left image", "Right image")

In [ ]:
imgs = [cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_4.jpg'), cv2.COLOR_BGR2RGB), 
        cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_5.jpg'), cv2.COLOR_BGR2RGB)]

### Stitch the images 

In [ ]:
def stitch_images(images):
    # Create a Stitcher object
    stitcher = cv2.Stitcher_create(cv2.Stitcher_PANORAMA)
    
    # Stitch the images together
    # output status, result image
    _ , result = stitcher.stitch(images)

    return result

### Result

In [ ]:
result = stitch_images(imgs)

plt.title("Result image")
plt.imshow(result)
plt.show()

## Image alignment for panorama using OpenCV function - 4 Images
(Detailed description in the Section 4.2.2 of the Report)

### Read the images

In [ ]:
# Load images:
img1 = cv2.imread('Photos/Bedroom/try1_1.jpg')
img2 = cv2.imread('Photos/Bedroom/try1_2.jpg')
img3 = cv2.imread('Photos/Bedroom/try1_3.jpg')
img4 = cv2.imread('Photos/Bedroom/try1_4.jpg')

# Convert the images to gray scale:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)

# Show the images
fig = plt.figure(figsize=(40, 80)) 
fig.add_subplot(1,4,1)
plt.title("Image 1")
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
fig.add_subplot(1,4,2)
plt.title("Image 2")
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
fig.add_subplot(1,4,3)
plt.title("Image 3")
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
fig.add_subplot(1,4,4)
plt.title("Image 4")
plt.imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
imgs = [cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_1.jpg'), cv2.COLOR_BGR2RGB), 
        cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_2.jpg'), cv2.COLOR_BGR2RGB),
        cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_3.jpg'), cv2.COLOR_BGR2RGB), 
        cv2.cvtColor(cv2.imread('Photos/Bedroom/try1_4.jpg'), cv2.COLOR_BGR2RGB)]

### Stitch the images

In [ ]:
result = stitch_images(imgs)

### Result

In [ ]:
plt.title("Result image")
plt.imshow(result)
plt.show()

## Image alignment for panorama using OpenCV function - 10 Images
(Detailed description in the Section 4.2.2 of the Report)

### Read the images

In [ ]:
# Load images:
imgs = []
for i in range(1, 11):
    file_name = 'Photos/Genova/3/img_{}.jpg'.format(i)
    imgs.append(cv2.imread(file_name))

# Convert the images to gray scale:
for i in range(len(imgs)):
    imgs[i] = cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY)

# plot the images:
num_rows, num_cols = 2, 5
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 8))
for i, ax in enumerate(axes.flatten()):
    # show image
    if i < len(imgs):
        ax.imshow(cv2.cvtColor(imgs[i], cv2.COLOR_BGR2RGB))
        ax.axis('off')
    # hide empty subplots
    else:
        ax.axis('off')

In [ ]:
imgs = []
for i in range(1, 11):
    file_name = 'Photos/Genova/3/img_{}.jpg'.format(i)
    imgs.append(cv2.cvtColor(cv2.imread(file_name), cv2.COLOR_BGR2RGB))

### Stitch the images

In [ ]:
result = stitch_images(imgs)

### Result

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.set_title("Result image")
ax.imshow(result)
plt.show()

## Optical Character Recognition (OCR)
(Detailed description in the Section 4.3 of the Report)

### Read the images

In [ ]:
book_cover = cv2.imread('Photos/Book/book_cover.jpeg')
book_test = cv2.imread('Photos/Book/img_2.jpg')

# Convert the images to gray scale:
book_cover = cv2.cvtColor(book_cover, cv2.COLOR_BGR2GRAY)
book_test = cv2.cvtColor(book_test, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(book_cover, book_test, "Book cover", "Test image")

### Localization and Description of Keypoints

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(book_cover, None)
kp2, des2 = sift.detectAndCompute(book_test, None)

print("Number of keypoints book cover:", len(kp1))
print("Number of keypoints test image:", len(kp2))

In [ ]:
# Draw the keypoints on the poster image:
sift_book_cover = None
sift_book_cover = cv2.drawKeypoints(book_cover, kp1, sift_book_cover, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_book/sift_book_cover.jpg', sift_book_cover)

# Draw the keypoints on the test image:
sift_book_test = None
sift_book_test = cv2.drawKeypoints(book_test, kp2, sift_book_test, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('Outputs/Outputs_form/sift_book_test.jpg', sift_book_test)

# Show the result of keypoint detection:
plot_2(sift_book_cover, sift_book_test, "Book cover keypoints", "Test image keypoints")

### Feature Matching

In [ ]:
good_list = []
good = []
good_list, good = featureMatching(des1, des2, good_list, good)

n_of_good_matches  = len(good)

In [ ]:
match_book = None
match_book = cv2.drawMatchesKnn(book_cover, kp1, book_test, kp2, good_list, match_book, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(cv2.cvtColor(match_book, cv2.COLOR_BGR2RGB))
plt.show()

### Find Homography

In [ ]:
homography = findHomography(n_of_good_matches, good, kp1, kp2)

### Warp perspective

In [ ]:
aligned_book = cv2.warpPerspective(book_test, homography, (book_cover.shape[1], book_cover.shape[0]))
cv2.imwrite('Outputs/Outputs_book/aligned_book.jpg', aligned_book)

### Alignment result

In [ ]:
# Show the result
plot_3(book_cover, book_test, aligned_book, "Book cover", "Test image", "Aligned book")

### Text Detection with Pytesseract 

In [ ]:
!pip3 install pytesseract
import pytesseract

In [ ]:
book_cover_text = pytesseract.image_to_string(aligned_book)
print(book_cover_text)

## Feature detectors comparison
(Detailed description in the Section 5 of the Report)

In [ ]:
# Load the input image
image = cv2.imread("Photos/Draw2/img1.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(image, cmap='gray')
plt.show()

### Time comparison

In [ ]:
import time

In [ ]:
# Compute the needed time for each feature detector:
start = time.time()
harris = cv2.cornerHarris(image, blockSize=2, ksize=3, k=0.04)
end = time.time()
harris_time = end - start

start = time.time()
shi_tomasi = cv2.goodFeaturesToTrack(image, maxCorners=10000, qualityLevel=0.01, minDistance=10)
end = time.time()
shi_tomasi_time = end - start

start = time.time()
sift = cv2.SIFT_create()
sift_keypoints, sift_descriptors = sift.detectAndCompute(image, None)
end = time.time()
sift_time = end - start

start = time.time()
orb = cv2.ORB_create()
orb_keypoints, orb_descriptors = orb.detectAndCompute(image, None)
end = time.time()
orb_time = end - start

start = time.time()
surf = cv2.xfeatures2d.SURF_create()
surf_keypoints, surf_descriptors = surf.detectAndCompute(image, None)
end = time.time()
surf_time = end - start

### Number of keypoints comparison

In [ ]:
# Calculate the number of detected features for each feature detector:
detected_features = [
    len(harris),
    len(shi_tomasi),
    len(sift_keypoints),
    len(orb_keypoints),
    len(surf_keypoints)
]

print('Harris keypoints:', len(harris))
print('Shi-Tomasi keypoints:', len(shi_tomasi))
print('SIFT keypoints:', len(sift_keypoints))
print('ORB keypoints:', len(orb_keypoints))
print('SURF keypoints:', len(surf_keypoints))

    
# Plot the chart
labels = ['Harris', 'Shi-Tomasi', 'SIFT', 'ORB', 'SURF']
plt.bar(labels, detected_features)
plt.xlabel('Feature Detectors')
plt.ylabel('Number of Detected Features')
plt.title('Comparison number of keypoints')
plt.show()

In [ ]:
detection_time = [("Harris time:", harris_time),
                  ("Shi-Tomasi time:", shi_tomasi_time),
                  ("SIFT time:", sift_time),
                  ("ORB time:", orb_time),
                  ("SURF time:", surf_time)]

for item in detection_time:
    print(item[0], item[1])

# Extract labels and times from the detection_time list
labels = [item[0] for item in detection_time]
times = [item[1] for item in detection_time]

# Plot the chart
plt.bar(labels, times)
plt.xlabel('Feature Detectors')
plt.ylabel("Time")
plt.title("Comparison Detection Time")
plt.xticks(rotation=45)
plt.show()

## Feature matcher comparison
(Detailed description in the Section 6 of the Report)

In [ ]:
# Load the input image
image1 = cv2.imread("Photos/Genova/1/img_3.jpg")
image2 = cv2.imread("Photos/Genova/1/img_4.jpg")

# Conver to gray
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Show the images
plot_2(image1, image2, "Image 1", "Image 2")

In [ ]:
# Initiate SIFT detector:
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT:
kp1, des1 = sift.detectAndCompute(image1, None)
kp2, des2 = sift.detectAndCompute(image2, None)

print(len(kp1))
print(len(kp2))

### Brute Force Matcher

In [ ]:
# BFMatcher with default params
start = time.time()
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)
end = time.time()
bf_time = end - start

# Apply ratio test
good_list = []
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good_list.append([m])
        good.append(m)

print("Number of good matches with BF Matcher:", len(good))

# cv.drawMatchesKnn expects list of lists as matches.
result_bf = cv2.drawMatchesKnn(image1, kp1, image2, kp2, good_list, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(result_bf)
plt.show()

### FLANN Matcher

In [ ]:
FLAN_INDEX_KDTREE = 0
index_params = dict (algorithm = FLAN_INDEX_KDTREE, trees=5)
search_params = dict (checks=50)

start = time.time()
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)
end = time.time()
flann_time = end - start

good_matches = []
for m1, m2 in matches:
  if m1.distance < 0.75 * m2.distance:
    good_matches.append([m1])

print("Number of good matches with FLANN Matcher:", len(good_matches))


flann_matches = cv2.drawMatchesKnn(image1, kp1, image2, kp2, good_matches, None, flags=2)
plt.figure(figsize = (50, 100))
plt.title("Matches")
plt.imshow(flann_matches)
plt.show()

### Time comparison

In [ ]:
matching_time = [("BF Matcher time:", bf_time),
                  ("FLANN Matcher time:", flann_time)]

for item in matching_time:
    print(item[0], item[1])

# Extract labels and times from the detection_time list
labels = [item[0] for item in matching_time]
times = [item[1] for item in matching_time]

# Plot the chart
plt.bar(labels, times)
plt.xlabel('Matcher')
plt.ylabel("Time")
plt.title("Comparison Matching Time")
plt.xticks(rotation=45)
plt.show()